### Prototype code to scrape the top headlines and their corresponding article text from News API

In [1]:
import os.path
from os import path
import pandas as pd
import json

In [2]:
import pprint
import requests

In [3]:
# API Key for News API
secret = 'd05c3ae08abd461f90b3dc555af1bb59'

In [4]:
# Define the endpoint to extract all the top headlines 
url = 'https://newsapi.org/v2/top-headlines?'

In [5]:
# Specify the query and number of returns - Limit the headlines to country US, for now
parameters = {
    'language': 'en',
    'country':'us',
    'pageSize': 100,
    'apiKey': secret 
}

In [6]:
categories = ['business','entertainment','general','health','science','sports','technology']


In [7]:
from newspaper import Article
from newspaper import Config


article_title = []
article_authors = []
article_text = []
article_summary = []
article_date = []
article_top_image = []
failed_url = []
category_articles ={}
pages= range(1,2)

In [8]:
#category_dataframe = pd.read_json(f'resources/business_top_headline_data.json').T
#category_dataframe.head()

In [9]:
count =0
for category in categories:
    name = category
    print(name)
    bappend = False
    if os.path.exists(f'resources/{category}_top_headline_data_new.json') :
        category_dataframe = pd.read_json(f'resources/{category}_top_headline_data_new.json').T
        bappend = True
    catDF= None
    parameters['category'] = category
    response = requests.get(url, params=parameters)
    if response.status_code != requests.codes.ok:
        print(f"Bad result : {response.url}")
        continue
    response_json = response.json()

    df = pd.DataFrame.from_dict(response_json)
    df = pd.concat([df.drop(['articles'], axis=1), df['articles'].apply(pd.Series)], axis=1) 

    rows = []
    for index, row in df.iterrows():
        h = row['url']
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
        config = Config()
        config.verbose=True
        config.browser_user_agent = user_agent
        newsarticle = Article(h, config=config)
        try :
            newsarticle.download()
            newsarticle.parse()
            if not newsarticle.text :
                print(f"Unable to parse article: {h}")
                continue 
            artdict = {}                
            artdict['articleText']=newsarticle.text
            artdict['articleSummary']=''   # Place Holder for bert summary
            artdict['articleSentiment']='' # Place Holder for calculated Sentiment Analysis
            for column in df:
                #print(column)
                if column in ['status', 'totalResults','content'] :
                    continue 
                artdict[column]= row[column]
            rows.append(artdict)
            count += 1
            if (count % 50 == 0 ):
                print('Number of articles: '+str(count))

        except :
             print(f'***FAILED TO DOWNLOAD***{newsarticle.url}')
             pass
    if name in category_articles:
        category_articles[name].append(pd.DataFrame(rows),ignore_index=True)
    else:
        if bappend :
            print(f"Using DataFrame {category_dataframe.shape} Adding {len(rows)}" )
            category_dataframe = category_dataframe.append(pd.DataFrame(rows),ignore_index=True).drop_duplicates(subset='url')
            category_articles[name] = category_dataframe
            print(category_articles[name].shape)        
        else:
            category_articles[name]= pd.DataFrame(rows)
    print(category_articles[name].shape)


print('Total number of articles: '+str(count))
    

business
***FAILED TO DOWNLOAD***https://www.wptv.com/news/region-c-palm-beach-county/west-palm-beach/template-west-palm-beach
Unable to parse article: https://www.profgalloway.com/lets-get-ready-to-rundle/
Number of articles: 50
***FAILED TO DOWNLOAD***https://news.google.com/__i/rss/rd/articles/CBMiXmh0dHBzOi8vd3d3LmFnd2ViLmNvbS9hcnRpY2xlL2Nvcm4tbWFya2V0LXVucGhhc2VkLXByb2R1Y3Rpb24tZHJvcC1sYXJnZS1jaGluZXNlLWJ1eS1oZXJlcy13aHnSAQA?oc=5
Using DataFrame (217, 10) Adding 67
(234, 10)
(234, 10)
entertainment
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9ZUJocUt6NFptNTTSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9WUJOWkY4My1JRGfSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9Y1VvZmtOVXlYRHPSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/artic

Unable to parse article: https://www.mmorpg.com/notice/ss-captcha
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9cW4wRHQ3NmdKYXPSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9ZGZQU3ZaeENuQXPSAQA?oc=5
Using DataFrame (198, 10) Adding 60
(211, 10)
(211, 10)
Total number of articles: 397


In [10]:
for category in category_articles.keys():
    print(category)
    df = category_articles[category]
    df.to_json(f'resources/{category}_top_headline_data_new.json', orient='index')
    
    

business
entertainment
general
health
science
sports
technology
